In [86]:
import toml
#Создаём файл конфигурации в формате TOML
data_dict = {
    "SMS_SERVICE": {
        "url": "http://localhost:4010",
        "username": "Valera",
        "password": "1234qwerty"
    }
}

with open('config.toml', 'w') as file:
    toml.dump(data_dict, file)

In [87]:
from typing import Self
#Создаём класс представляющий HTTP-запрос
class HTTPRequest:
    def __init__(self, method, url, headers, body):
        self.method = method
        self.url = url
        self.headers = headers
        self.body = body

    def to_bytes(self) -> bytes:
        headers = "\r\n".join(f"{k}: {v}" for k, v in self.headers.items())
        #print(f"{self.method} {self.url} HTTP/1.1\r\n{headers}\r\n\r\n{self.body}")
        return f"{self.method} {self.url} HTTP/1.1\r\n{headers}\r\n\r\n{self.body}".encode()

#Создаём класс представляющий HTTP-ответ  
class HTTPResponse:
    def __init__(self, status_code, headers, body):
        self.status_code = status_code
        self.headers = headers
        self.body = body

    @classmethod
    def from_bytes(cls,binary_data: bytes) -> Self:
        # Реализация преобразования байт в объект HTTPResponse
        headers_part, body = binary_data.split(b"\r\n\r\n", 1)
        
        headers = headers_part.decode().split("\r\n")
        status_line = headers[0]
        status_code = status_line.split(" ", 2)
        
        header_dict = {}
        for line in headers[1:]:
            if line:
                key, value = line.split(": ", 1)
                header_dict[key] = value
        body=body.decode()
        return cls(status_code,header_dict,body)


In [88]:
import logging
#Логирование в файл
logging.basicConfig(level=logging.INFO, filename="py_log.log",filemode="w",
                    format="%(asctime)s %(levelname)s %(message)s")

In [89]:
import socket
#Реализация передачи HTTP-запроса и получения HTTP-ответа
def send_request(request: HTTPRequest) -> HTTPResponse:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect(("localhost", 4010))
            s.sendall(request.to_bytes())
            data = s.recv(4096)
        return HTTPResponse.from_bytes(data)
    except socket.error as e:
        logging.error(f"Socket error: {e}")
        raise

In [90]:
import toml
#Загружаем данные из файла конфигурации
def load_config():
    with open('config.toml','r') as f:
        return toml.load(f)

In [94]:
import json
import base64
def main():
    print("Enter the sender of the recipient and the message\r\n")
    data=input()
    try:
        data_split=data.split(' ',2)
        if len(data_split) != 3:
            raise ValueError("Invalid input format. Expected: sender recipient message")
        sender, recipient, message = data_split
        if not sender or not recipient or not message:
            logging.error("One of the fields is empty")
        else:
            logging.info(f"Sender: {sender}, Recipient: {recipient}, Message: {message}")
    except ValueError:
        logging.error("An empty line has been entered")
    
    
    config = load_config()
    sms_service = config["SMS_SERVICE"]

    request = HTTPRequest(
        method="POST",
        url="/send_sms",
        headers={
            "Host": "localhost:4010",
            "Authorization": f"Basic {base64.b64encode(f'{sms_service['username']}:{sms_service['password']}'.encode()).decode()}",
            "Content-Type": "application/json"
        },
        body=json.dumps({
            "sender": data_split[0],
            "recipient": data_split[1],
            "message": data_split[2]
        })
        
    )

    response = send_request(request)
    headers= "\r\n".join(f"{k}: {v}" for k, v in request.headers.items())
    logging.info(f"HTTP-Request:\r\n{request.method} {request.url} HTTP/1.1\r\n{headers}\r\n\r\n{request.body}\r\n")
    logging.info(f"HTTP-Response:\r\n{response.status_code}\r\n{response.body}")
    print(f"Status Code: {response.status_code}")
    print(f"Response Body: {response.body}")

main()

Enter the sender of the recipient and the message



 99 98 


Status Code: ['HTTP/1.1', '400', 'Bad Request']
Response Body: 
